In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')

data.head()

In [ ]:
data.drop(columns=['sl_no','salary'],inplace=True)
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
target = data.loc[:,'status']
data = data.iloc[:,0:-1]

In [ ]:
target

In [ ]:
data

In [ ]:
nom_cols = [0,5,7,10]

ord_cols = [2,4,8]

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
trans_data = make_column_transformer((OneHotEncoder(sparse=False),nom_cols),(OrdinalEncoder(),ord_cols),remainder = "passthrough")

In [ ]:
trans_data

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(data,target,test_size=0.3)

In [ ]:
data.shape

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_test.shape

In [ ]:
# from sklearn.model_selection import KFold 

# kf = KFold(n_splits=3)

# kf

In [ ]:
# for train_index , test_index in kf.split([0,1,2,3,4,5,6,7,8]):
#     print(train_index, test_index)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors = 5)

model

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn import set_config

set_config(display='diagram')

pipe = make_pipeline(trans_data,model)

pipe

In [ ]:
pipe.fit(x_train,y_train)

In [ ]:
pred = pipe.predict(x_test)

pred

In [ ]:
y_test

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(pred,y_test)*100

In [ ]:
pipe.predict(pd.DataFrame(np.array(['F',65,'Central',70,'Others','Arts',50,'Comm&Mgmt','No',74,'Mkt&Fin',80]).reshape(1,12)))

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

s_fold = StratifiedKFold(n_splits=5, shuffle=True,random_state=3)

np.mean(cross_val_score(pipe,data,target,cv = s_fold, scoring='accuracy'))*100

In [ ]:
s_fold

In [ ]:
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(pipe, x_test, y_test, display_labels=['Placed','Not Placed'],cmap="Blues")

In [ ]:
from sklearn.metrics import precision_score,recall_score

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver = "liblinear")

log_reg_pipe = make_pipeline(trans_data, lr)

log_reg_pipe

In [ ]:
log_reg_pipe.fit(x_train,y_train)

In [ ]:
pred = log_reg_pipe.predict(x_test)
pred

In [ ]:
accuracy_score(pred,y_test)*100

In [ ]:
import seaborn as sns
sns.countplot(target)

In [ ]:
target.value_counts()

In [ ]:
import imblearn

#!pip install imbalanced-learn

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

under = RandomUnderSampler()

x_under,y_under = under.fit_resample(data,target)

In [ ]:
sns.countplot(y_under)

In [ ]:
y_under.value_counts()

In [ ]:
from imblearn.over_sampling import RandomOverSampler

over = RandomOverSampler()

x_over,y_over = over.fit_resample(data,target)

In [ ]:
sns.countplot(y_over)

In [ ]:
y_over.value_counts()

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()

x_smote, y_smote = smote.fit_resample(trans_data.transform(data),target)

In [ ]:
sns.countplot(y_smote)

In [ ]:
from imblearn.pipeline import make_pipeline


pipe_smote = make_pipeline(trans_data,smote,lr)

pipe_smote

In [ ]:
pipe_smote.fit(x_train,y_train)

In [ ]:
accuracy_score(pipe_smote.predict(x_test),y_test)*100

Apply StratifiedKFold and obtain mean accuracy using cross_val_score

In [ ]:
np.mean(cross_val_score(pipe_smote,data,target,cv = s_fold, scoring='accuracy'))*100

In [ ]:
from sklearn.svm import SVC

model_svc = SVC(kernel = 'rbf')  #linear,poly,rbf

pipe_svc = make_pipeline(trans_data,smote,model_svc)

pipe_svc.fit(x_train,y_train)

In [ ]:
accuracy_score(pipe_svc.predict(x_test),y_test)*100